Q4 (linechart)
county, state, shootings, year

In [144]:
import pandas as pd
import altair as alt

In [ ]:
df_shootings = pd.read_csv("datasets/MassShootingsComplete_FIPS.csv")
df_population = pd.read_csv("datasets/CountyPopulationAllYears.csv")

In [413]:
'''
CHART 1: total shootings / year   +   governement (democratic / republican)
'''

df_shootings = pd.read_csv("datasets/MassShootingsComplete_FIPS.csv")
gov = pd.DataFrame({
    'Year': [i for i in range(2014, 2025)],
    'y1': [0]*11, 
    'y2': [0]*11,
    'governement': ['Democratic']*3 + ['Republican']*5 + ['Democratic']*3
})


chart_gov = alt.Chart(gov).mark_area().encode(
    x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('y1:Q', title='Total Shootings', scale=alt.Scale(domain=(200, 720))),
    y2='y2:Q',
    color=alt.Color('governement', legend=alt.Legend(
        orient='top-left', 
        legendX=130, legendY=-100,  # Ajusta la posició de la llegenda
        strokeColor='gray',
        fillColor='#EEEEEE',
        padding=10,
        cornerRadius=10,
        titleAnchor='middle'  # Centra el títol dins de la llegenda
    ), scale=alt.Scale(domain=['Democratic', 'Republican'], range=['#3182bd', '#d7301f']))
)

# Converteix la data a un format de data de pandas
df_shootings['Incident Date'] = pd.to_datetime(df_shootings['Incident Date'])

# Extreu l'any de la data de l'incident
df_shootings['Year'] = df_shootings['Incident Date'].dt.year

# Agrupa per any i compta el nombre de tirotejos
shootings_per_year = df_shootings.groupby('Year').size().reset_index(name='Total Shootings')

# Crea el gràfic de línia amb Altair
chart_ms = alt.Chart(shootings_per_year).mark_line(
    color='black',
    point=alt.OverlayMarkDef(filled=True, fill="black"),
    strokeWidth=2.5).encode(
    x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=0,grid=False)),  # Usa ':O' per especificar que l'eix X és ordinal
    y=alt.Y('Total Shootings:Q', title='Total Shootings').scale(domain=(250,710))
).properties(
    title='Total Shootings per Year in the USA',
    width=500,
    height=400
)


rep = alt.Chart().mark_rect(color=' #f5b7b1  ', opacity=1).encode(
    x=alt.value(160),
    y=alt.value(0),
    x2=alt.value(341),
    y2=alt.value(400))

dem1 = alt.Chart().mark_rect(color=' #aed6f1  ', opacity=1).encode(
    x=alt.value(0),
    y=alt.value(0),
    x2=alt.value(160.5),
    y2=alt.value(400))

dem2 = alt.Chart().mark_rect(color=' #aed6f1  ', opacity=1).encode(
    x=alt.value(341.2),
    y=alt.value(0),
    x2=alt.value(500),
    y2=alt.value(400))

# Background rectangles
background = alt.layer(
    dem1, rep, dem2
)

y_values = pd.DataFrame({'y': list(range(0, 701, 50))})

# Crear les línies utilitzant mark_rule
lines = alt.Chart(y_values).mark_rule(color='black', size=0.8, opacity=0.5).encode(
    y=alt.Y('y:Q',title=None, axis=None)
)

# Line charts layered on top of the background rectangles
chart_with_background = alt.layer(
    background,
    lines,
    chart_ms,
    chart_gov
).properties(
    title='Total Shootings per Year in the USA',
    width=500,
    height=400
).resolve_scale(
    y='independent'
)



# Crear una configuració per a l'eix
chart_with_background = chart_with_background.configure_axisX(
    grid=False  # Desactiva les línies de la graella verticals
).configure_axisY(
    grid=True,         # Activa les línies de la graella horitzontals
    gridColor='grey', # Defineix el color de les línies horitzontals
    gridWidth=1.1
)

chart_with_background

alt.LayerChart(...)

In [414]:
'''
CHART 2: total shootings / year and / county
'''


shootings_per_state_year = df_shootings.groupby(['State', 'Year']).size().reset_index(name='Total Shootings')

chart = alt.Chart(shootings_per_state_year).mark_line().encode(
    x=alt.X('Year:O', title='Year'),  # Usa ':O' per especificar que l'eix X és ordinal
    y=alt.Y('Total Shootings:Q', title='Total Shootings'),
    color='State:N'  # Utilitza 'State' per distingir cada línia amb un color diferent
).properties(
    title='Total Shootings per Year per State in the USA',
    width=500,
    height=400
)

chart.show()

#not feasable... xdd

alt.Chart(...)

In [155]:
years = range(2014, 2024)

# Comptar el total de tirotejos per cada any
df_shootings['Incident Date'] = pd.to_datetime(df_shootings['Incident Date'])
df_shootings['Year'] = df_shootings['Incident Date'].dt.year
shootings_totals = df_shootings.groupby("Year").size().reindex(years, fill_value=0)

# Calcular la població total per cada any al dataset de població
population_totals = df_population[[f"POPESTIMATE{year}" for year in years]].sum()

# Calcular la ràtio tirotejos/població per cada any
ratio = shootings_totals.values * 1e6 / population_totals.values

# Crear un DataFrame amb els resultats
data = pd.DataFrame({
    'Year': years,
    'Shootings per capita': ratio
})

chart_ms = alt.Chart(data).mark_line(
    color='black',
    point=alt.OverlayMarkDef(filled=True, fill="black"),
    strokeWidth=2.5).encode(
    x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Shootings per capita:Q', title='Total Shootings', scale=alt.Scale(domain=(0.3,1.05)))
).properties(
    title='Shootings per Capita *10e6  in the USA',
    width=500,
    height=400
)

gov = {
    'Year': [i for i in range(2014, 2025)],
    'y1': [0.3+0.042]*11,
    'y2': [0.4-0.042]*11,
    'governement': ['Democratic']*3 + ['Republican']*5 + ['Democratic']*3
}

# Convert to a DataFrame
df = pd.DataFrame(gov)

# Create the Altair bar chart using 'y1' or 'y2' as the Total Shootings value
chart_gov = alt.Chart(df).mark_area().encode(
    x=alt.X('Year:O', title='Year', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('y1:Q', title='Total Shootings'),
    y2='y2:Q',  # Specify the y2 encoding for the upper boundary of the area
    color=alt.Color('governement', scale=alt.Scale(domain=['Democratic', 'Republican'], range=['#3182bd', '#d7301f']))
).properties(
    title='Total Shootings per Year in the USA',
    width=500,
    height=400
)

linechart_years = chart_ms + chart_gov

In [156]:
df_shootings['Incident Date'] = pd.to_datetime(df_shootings['Incident Date'])

# Afegir columnes d'any i mes
df_shootings['Year'] = df_shootings['Incident Date'].dt.year
df_shootings['Month'] = df_shootings['Incident Date'].dt.month



# Agrupar les dades per any i mes, i comptar el nombre de tirotejos
monthly_shootings = df_shootings.groupby(['Year', 'Month']).size().reset_index(name='Total Shootings')

line_chart = alt.Chart(monthly_shootings[monthly_shootings['Year']>2013]).mark_line().encode(
    x=alt.X('Month:O', title='Mes', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Total Shootings:Q', title='Nombre total de tirotejos'),
    color=alt.Color('Year:O', title='Any')
).properties(
    title='Evolució mensual dels tirotejos massius als EUA per any'
)

line_chart

alt.Chart(...)

In [157]:
monthly_shootings['year-month'] = (monthly_shootings['Year'])*100 + (monthly_shootings['Month']-1)*10/11

line_chart = alt.Chart(monthly_shootings[monthly_shootings['Year']>2013]).mark_line().encode(
    x=alt.X('year-month:O', title='Mes', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Total Shootings:Q', title='Nombre total de tirotejos')
).properties(
    title='Evolució mensual dels tirotejos massius als EUA per any'
)

line_chart

alt.Chart(...)

In [132]:
((monthly_shootings['Month']-1)*10/11).head(20)

0      0.000000
1      0.909091
2      1.818182
3      2.727273
4      3.636364
5      4.545455
6      5.454545
7      6.363636
8      7.272727
9      8.181818
10     9.090909
11    10.000000
12     0.000000
13     0.909091
14     1.818182
15     2.727273
16     3.636364
17     4.545455
18     5.454545
19     6.363636
Name: Month, dtype: float64

In [83]:
df_shootings['Incident Date'] = pd.to_datetime(df_shootings['Incident Date'])

# Afegir columnes d'any i mes
df_shootings['Year'] = df_shootings['Incident Date'].dt.year
df_shootings['Month'] = df_shootings['Incident Date'].dt.month



# Agrupar les dades per any i mes, i comptar el nombre de tirotejos
monthly_shootings = df_shootings.groupby(['Year', 'Month']).size().reset_index(name='Total Shootings')

line_chart = alt.Chart(monthly_shootings[monthly_shootings['Year']>2019]).mark_line().encode(
    x=alt.X('Month:O', title='Mes', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Total Shootings:Q', title='Nombre total de tirotejos'),
    color=alt.Color('Year:N', title='Any')
).properties(
    title='Evolució mensual dels tirotejos massius als EUA per any'
)

line_chart

alt.Chart(...)

In [158]:
import pandas as pd
import altair as alt

# Calcular la mitjana de tirotejos per mes
monthly_shootings = df_shootings[df_shootings['Year']>2014].groupby(['Year', 'Month']).size().reset_index(name='Total Shootings')
monthly_avg = monthly_shootings.groupby('Month')['Total Shootings'].mean().reset_index()

# Afegir els noms dels mesos en català
mesos = ['January', 'February', 'March', 'April', 'May', 'June', 
          'July', 'August', 'September', 'October', 'November', 'December']
monthly_avg['Mes'] = [mesos[m-1] for m in monthly_avg['Month']]

# Crear el gràfic base
base = alt.Chart(monthly_avg).encode(
    alt.Theta("Month:O").stack(True),
    alt.Radius("Total Shootings").scale(type="sqrt", zero=True, rangeMin=20),
    alt.Color('Total Shootings:Q', scale=alt.Scale(scheme='reds'))  
)

# Crear els arcs
c1 = base.mark_arc(innerRadius=20, stroke="#fff")

# Afegir les etiquetes amb els noms dels mesos
c2 = base.mark_text(
    radiusOffset=18,
    color='black'  # Aquí especifiquem el color negre
).encode(
    text="Mes:N",
    color=alt.condition(
        "datum.Total_Shootings > " + str(5000),
        alt.value("white"),
        alt.value("black"),
    )
)

# Combinar els dos gràfics i configurar
final_chart = (c1 + c2).properties(
    width=400,
    height=400,
    title="Mitjana mensual de tirotejos massius als EUA"
).configure_legend(
    disable=True  # Amaguem la llegenda ja que tenim les etiquetes
)

final_chart

alt.LayerChart(...)

In [159]:
import pandas as pd
import altair as alt

# Calcular la mitjana de tirotejos per mes
monthly_avg = monthly_shootings.groupby('Month')['Total Shootings'].mean().reset_index()

# Afegir els noms dels mesos en anglès
months = ['January', 'February', 'March', 'April', 'May', 'June', 
          'July', 'August', 'September', 'October', 'November', 'December']
monthly_avg['Month Name'] = [months[m-1] for m in monthly_avg['Month']]

# Crear el heatmap en format vertical
heatmap = alt.Chart(monthly_avg).mark_rect().encode(
    y=alt.Y('Month Name:N', sort=months, title=None),  # Canviar a l'eix Y
    color=alt.Color('Total Shootings:Q',
                    scale=alt.Scale(scheme='reds'),
                    legend=None,  # Treure la llegenda
                    title='Average Shootings'),
    tooltip=[
        alt.Tooltip('Month Name:N', title='Month'),
        alt.Tooltip('Total Shootings:Q', title='Average', format='.1f')
    ]
).properties(
    width=50,  # Ajustar l'amplada
    height=400,  # Ajustar l'alçada
    title='Average Distribution of Mass Shootings per Month'
)

# Afegir els valors numèrics sobre cada cel·la
text = alt.Chart(monthly_avg).mark_text(
    baseline='middle',
    color='white'
).encode(
    y=alt.Y('Month Name:N', sort=months),
    text=alt.Text('Total Shootings:Q', format='.1f')
)

# Combinar el heatmap amb el text
heat_map_month_distribution = (heatmap + text).configure_axis(
    labelAngle=0,  # Mantenir les etiquetes horitzontals
    labelFontSize=12
).configure_title(
    fontSize=14,
    anchor='middle'
)

heat_map_month_distribution


alt.LayerChart(...)